In [1]:

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_05b import *
torch.set_num_threads(2)

C:\ProgramData\Anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
x_train,y_train,x_valid,y_valid = get_data()

In [6]:
#export
def normalize_to(train, valid):
    m,s = train.mean(),train.std()
    return normalize(train, m, s), normalize(valid, m, s)

In [7]:
x_train,x_valid = normalize_to(x_train,x_valid)
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

In [9]:

x_train.mean(),x_train.std()

(tensor(-1.6608e-09), tensor(1.0000))

In [10]:
nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [14]:
#export
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x): return self.func(x)

def flatten(x):      return x.view(x.shape[0], -1)

In [20]:
def mnist_resize(x):
    return x.view(-1,1,28,28)

In [24]:
def get_cnn_model(data):
    return nn.Sequential(
        Lambda(mnist_resize),
        nn.Conv2d( 1, 8, 5, padding=2,stride=2), nn.ReLU(), #14
        nn.Conv2d( 8,16, 3, padding=1,stride=2), nn.ReLU(), # 7
        nn.Conv2d(16,32, 3, padding=1,stride=2), nn.ReLU(), # 4
        nn.Conv2d(32,32, 3, padding=1,stride=2), nn.ReLU(), # 2
        nn.AdaptiveAvgPool2d(1),
        Lambda(flatten),
        nn.Linear(32,data.c)
    )

In [28]:
model = get_cnn_model(data)

In [32]:
cbfs = [Recorder, partial(AvgStatsCallback,accuracy)]

In [36]:
opt = optim.SGD(model.parameters(), lr=0.4)
learn = Learner(model, opt, loss_func, data)
run = Runner(cb_funcs=cbfs)

In [40]:
run.fit(5,learn)

train: [0.3111564453125, tensor(0.9061)]
valid: [0.2239073974609375, tensor(0.9314)]
train: [0.152398486328125, tensor(0.9545)]
valid: [0.2969731201171875, tensor(0.9124)]
train: [0.11368599446614583, tensor(0.9669)]
valid: [0.1783311767578125, tensor(0.9433)]
train: [0.12972578125, tensor(0.9604)]
valid: [0.09655089721679687, tensor(0.9699)]
train: [0.07239269205729167, tensor(0.9776)]
valid: [0.07280846557617188, tensor(0.9767)]


In [45]:
# Somewhat more flexible way
device = torch.device('cuda',0)

In [46]:

class CudaCallback(Callback):
    def __init__(self,device): self.device=device
    def begin_fit(self): self.model.to(self.device)
    def begin_batch(self): self.run.xb,self.run.yb = self.xb.to(self.device),self.yb.to(self.device)

In [47]:
# Somewhat less flexible, but quite convenient
torch.cuda.set_device(device)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [48]:
#export
class CudaCallback(Callback):
    def begin_fit(self): self.model.cuda()
    def begin_batch(self): self.run.xb,self.run.yb = self.xb.cuda(),self.yb.cuda()